# Conhecendo os dados


In [ ]:
import pandas as pd

pd.__version__

In [ ]:
url = 'pandas-dados/dataset-telecon.json'

In [ ]:
dados_churn = pd.read_json(url)
dados_churn.head()

In [ ]:
dados_churn[0:1].to_json()

In [ ]:
dados_churn.iloc[0]

In [ ]:
pd.json_normalize(dados_churn['conta']).head()

In [ ]:
pd.json_normalize(dados_churn['telefone']).head()

## Transformando dados em uma tabela


Pude perceber que utilizar o json_normalize() para todas colunas é mais trabalho, portanto, deve existir um jeito mais facil de normalizar o JSON de um vez só, sem a necessidade de fazer a normalizacao uma para cada coluna.


In [ ]:
import json

In [ ]:
with open(url) as f:
    json_bruto = json.load(f)

In [ ]:
json_bruto

In [ ]:
dados_normalizados = pd.json_normalize(json_bruto)
dados_normalizados.head()


In [ ]:
dados_normalizados.iloc[0].to_json()

## Transformação inicial dos dados


### Entendendo os dados

A base de dados contém colunas além do ID dos clientes e o churn:

**Cliente:**

- `genero:` gênero (masculino e feminino)
- `idoso:` informação sobre um cliente ter ou não idade igual ou maior que 65 anos
- `parceiro:` se o cliente possui ou não um parceiro ou parceira
- `dependentes:` se o cliente possui ou não dependentes
- `tempo_servico:` meses de contrato do cliente

**Serviço de telefonia**

- `servico_telefone:` assinatura de serviço telefônico
- `varias_linhas:` assinatura de mais de uma linha de telefone

**Serviço de internet**

- `servico_internet:` assinatura de um provedor internet
- `seguranca_online:` assinatura adicional de segurança online
- `backup_online:` assinatura adicional de backup online
- `protecao_dispositivo:` assinatura adicional de proteção no dispositivo
- `suporte_tecnico:` assinatura adicional de suporte técnico, menos tempo de espera
- `tv_streaming:` assinatura de TV a cabo
- `filmes_streaming:` assinatura de streaming de filmes

**Conta**

- `contrato:` tipo de contrato
- `faturamente_eletronico:` se o cliente prefere receber online a fatura
- `metodo_pagamento:` forma de pagamento
- `cobranca.mensal:` total de todos os serviços do cliente por mês
- `cobranca.Total:` total gasto pelo cliente


In [ ]:
dados_normalizados.info()

### Modificando o tipo de coluna


In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].head()

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '][['cliente.tempo_servico','conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
idx = dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].index
idx

In [ ]:
dados_normalizados.loc[idx, 'conta.cobranca.Total'] = dados_normalizados.loc[idx, 'conta.cobranca.mensal'] * 24

In [ ]:
dados_normalizados.loc[idx, 'cliente.tempo_servico'] = 24

In [ ]:
dados_normalizados.loc[idx][['cliente.tempo_servico','conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
dados_normalizados.info()

vamos visualizar os valores unicos de cada coluna


In [ ]:
for col in dados_normalizados.columns:
    print(f'Coluna: {col}')
    print(dados_normalizados[col].unique())
    print('-' * 30)

In [ ]:
# verifiquei que na coluna Churn tem um ' ' que nao diz respeito e nao ajuda em analise alguam, portanto, vamos tratar isso
dados_normalizados.query("Churn == ''")

In [ ]:
# vamos descartar essas informacoes de cima porque nao faz sentido ficarem em branco

dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()

In [ ]:
dados_sem_vazio.info()

In [ ]:
# como podemos perceber temos index faltantes e precisamos ajustar isso (Index: 7118 entries, 0 to 7343), pois em um modelo de machine learning ele espera que vamos ter indexes consecutivos
# drop = True vai retirar a coluna index que ele iria adicionar, o inplace já altera automaticamente sem precisar referenciar.
dados_sem_vazio.reset_index(drop=True, inplace=True)
dados_sem_vazio

## Trabalhando com dadso duplicados e nulos


### Identificando e tratando dados duplicados

dados duplicados ==> possuem os mesmos valores em colunas correspondentes


In [ ]:
# o metodo duplicated retorna uma series booleana. false para quando a amostra nao é duplicada, e True para quando a amostra é duplicada
dados_sem_vazio.duplicated().value_counts()

In [ ]:
dados_sem_vazio.duplicated().sum()

In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated()
filtro_duplicadas

In [ ]:
dados_sem_vazio[filtro_duplicadas]

Portanto, para essas amostras duplicadas, vamos precisar remove-las do conjunto, pois:

- Vies do modelo: pode ser que o modelo dë mais importancia para essas amostras que sao duplicadas
- Melhora do desempenho do modelo: se inserimos amostra que sao duplicadas, estaremos desperdicando poder computacional, para processar com amostras que transmitem o mesmo valor
- Aumento da qualidade dos resultados: isso porque estaremos utilizando informacoes que transmitem mais qualidade por serem unicas e relevantes


In [ ]:
dados_sem_vazio.drop_duplicates(inplace=True)

In [ ]:
dados_sem_vazio

In [ ]:
dados_sem_vazio.duplicated().sum()

### Identificando e tratando dados nulos

dados nulos ==> valores ausentes ou desconhecidos na base de dados


In [ ]:
# se o valor for False é porque nao é nulo
# se o valor do True é porque o valor é nulo
dados_sem_vazio.isna()

In [ ]:
# da forma anterior é ruim de verificar a quantidade de informacoes nulas e portanto conseguimos usar o codigo abaixo para verificar onde temos dados nulos
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio.isna().sum().sum()

In [ ]:
# vamos fazer um filtro para entender quantas amostras temos nulas
# o axis = 1 faz referencia a coluna
dados_sem_vazio[dados_sem_vazio.isna().any(axis=1)]

In [ ]:
dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
# analisando as colunas que temos dados nulos, podemos ajustar a tempo_servico porque ela possui uma relacao com as demais
filtro = dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico','conta.cobranca.mensal','conta.cobranca.Total']]

In [ ]:
idx_vazio = dados_sem_vazio[filtro].index
idx_vazio

In [ ]:
import numpy as np

In [ ]:
dados_sem_vazio.loc[idx_vazio, 'cliente.tempo_servico'] = np.ceil(dados_sem_vazio.loc[idx_vazio, 'conta.cobranca.Total'] / dados_sem_vazio.loc[idx_vazio, 'conta.cobranca.mensal'])
dados_sem_vazio[filtro][['cliente.tempo_servico','conta.cobranca.mensal','conta.cobranca.Total']]

##### poderiamos utilizar essa segunda abortagem tambem para fazer essa conta anterior

```python
dados_sem_vazio['cliente.tempo_servico'].fillna(
    np.ceil(
        dados_sem_vazio['conta.cobranca.Total'] / dados_sem_vazio['conta.cobranca.mensal']
    ), inplace= True
)

```


In [ ]:
dados_sem_vazio.isna().sum()

podemos perceber que todas que ainda tem valores nulos, vem da chave conta. Será que podemos realizar algum tipo de inserção igual fizemos anteriormente?


In [ ]:
dados_sem_vazio['conta.contrato'].value_counts()

In [ ]:
# jã que nao tenho como inferir algo para substituir os valores, vamos retirar essas informacoes

colunas_dropar = ['conta.contrato', 'conta.faturamente_eletronico', 'conta.metodo_pagamento']

In [ ]:
dados_sem_vazio[colunas_dropar].isna()

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis=1)

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis=1).sum()

In [ ]:
# portanto temos 37 amostras que tem pelo menos um valor nulo nessas tres colunas

df_sem_nulo = dados_sem_vazio.dropna(subset=colunas_dropar).copy()
df_sem_nulo.head()

In [ ]:
# vamos reiniciar o indeces porque nao esta iniciando em zero

df_sem_nulo.reset_index(drop = True, inplace=True)

In [ ]:
df_sem_nulo.head()

In [ ]:
df_sem_nulo.isna().sum()

## Para saber mais: inserindo a moda nos dados

A imputação de valores nulos em um Pandas DataFrame é uma das técnicas úteis quando se trabalha com dados faltantes. A imputação é o processo de substituir os valores faltantes por outros valores, de modo a tornar o conjunto de dados mais completo e utilizável.

A moda é uma medida estatística que representa o valor mais frequente em um conjunto de dados. Utilizá-la para preencher valores nulos é uma técnica simples e eficaz que pode ajudar a melhorar a qualidade dos dados.

Imagine que você esteja trabalhando com um conjunto de dados que contém informações sobre vendas em uma loja. Algumas dessas informações podem estar faltando, como o tamanho de alguns produtos. Nesse caso, você pode usar a moda para preencher os valores nulos. Isso significa que você irá substituir os valores faltantes pelo tamanho mais comum dos produtos.

Para fazer isso no Pandas, você pode seguir conforme código abaixo:

```python
import pandas as pd

# Criando um DataFrame de exemplo

df = pd.DataFrame({'Produto': ['Camisa', 'Calça', 'Tênis', 'Meia', 'Boné'],
                   'Tamanho': ['P', 'M', 'M', None, None],
                   'Preço': [49.99, 79.99, 199.99, 9.99, 39.99]})

# Preenchendo os valores nulos com a moda

df['Tamanho'].fillna(df['Tamanho'].mode()[0], inplace=True)
print(df)
```

Nesse exemplo, o DataFrame possui valores nulos na coluna "Tamanho". Para preencher esses valores com a moda, utilizamos o método "fillna()" em conjunto com o método "mode()". Esta retorna o valor mais comum em um conjunto de dados.

No caso do DataFrame acima, a moda da coluna "Tamanho" é "M". Portanto, utilizamos esse valor para preencher os valores nulos na respectiva coluna. O resultado final é um DataFrame mais completo e utilizável, pronto para ser analisado e usado em futuras análises e decisões de negócios.

Ao executar o código acima, o resultado será o seguinte DataFrame:

<table>
    <thead>
        <tr>
            <th></th>
            <th>Produto</th>
            <th>Tamanho</th>
            <th>Preço</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0</td>
            <td>Camisa</td>
            <td>P</td>
            <td>49.99</td>
        </tr>
        <tr>
            <td>1</td>
            <td>Calça</td>
            <td>M</td>
            <td>79.99</td>
        </tr>
        <tr>
            <td>2</td>
            <td>Tênis</td>
            <td>M</td>
            <td>199.99</td>
        </tr>
        <tr>
            <td>3</td>
            <td>Meia</td>
            <td>M</td>
            <td>9.99</td>
        </tr>
        <tr>
            <td>4</td>
            <td>Boné</td>
            <td>M</td>
            <td>39.99</td>
        </tr>
    </tbody>
</table>

Observe que os valores nulos foram substituídos pela moda.

Para conferir com mais detalhes os métodos utilizados, você pode consultar suas documentações oficiais:

- [método mode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html)
- [método fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)


## Trabalhando com os outliers


#### Outlier

- Valor atipico ou ponto fora da curva
- Dado que difere do padrao do conjunto de dados significativamente


##### Identificando os outliers


In [ ]:
df_sem_nulo.describe()

In [ ]:
# iremos utilizar o blox plot para entender se temos algum outlier na coluna de tempo_servico, porque 1080 meses sao 90 anos e é muito dificil alguem assinar por tanto tempo. 
import seaborn as sns

In [ ]:
sns.boxplot(x=df_sem_nulo['cliente.tempo_servico'])

In [ ]:
# pela figura anterior, podemos perceber que temos 10 itens que sao outliers, logo temos que tentar identificar quais sao
Q1 = df_sem_nulo['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_nulo['cliente.tempo_servico'].quantile(.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR
print(f'Q1: {Q1}\nQ3: {Q3}\nIQR: {IQR}\nlimite_inferior: {limite_inferior}\nlimite_superior: {limite_superior}\n')

In [ ]:
# portanto, com isso conseguimos aplicar o filtro de valores menores que o limite inferior ou maiores que o limite superior

outliers_index = (df_sem_nulo['cliente.tempo_servico'] < limite_inferior) | (df_sem_nulo['cliente.tempo_servico'] > limite_superior)
outliers_index

In [ ]:
df_sem_nulo[outliers_index]['cliente.tempo_servico']

##### Substituindo valores para os Outliers


In [ ]:
df_sem_out = df_sem_nulo.copy()

In [ ]:
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal','conta.cobranca.Total']]

In [ ]:
df_sem_out.loc[outliers_index, 'cliente.tempo_servico'] = np.ceil(
    df_sem_out.loc[outliers_index, 'conta.cobranca.Total'] / df_sem_out.loc[outliers_index, 'conta.cobranca.mensal']
)

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
# percebemos que ainda temos alguns valores que sao outliers, portanto, vamos verificar quais sao
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal','conta.cobranca.Total']]

**`mesmo apos realizarmos a atribuicao dos valores, percebemos que de fato nao podemos atribuir com exatidao e certeza os valores da coluna 'cliente.tempo_servico', logo, o mais aconselhavél é retirar essas informacoes do conjunto de dados que estamos preparando para um modelo de machine learning`**


##### **Removendo Outliers**


In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
Q1 = df_sem_out['cliente.tempo_servico'].quantile(.25)
Q3 = df_sem_out['cliente.tempo_servico'].quantile(.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5*IQR
limite_superior = Q3 + 1.5*IQR

outliers_index = (df_sem_out['cliente.tempo_servico'] < limite_inferior) | (df_sem_out['cliente.tempo_servico'] > limite_superior)
outliers_index

In [ ]:
df_sem_out[outliers_index]

In [ ]:
df_sem_out = df_sem_out[~outliers_index]
df_sem_out

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out.reset_index(drop=True, inplace=True)

In [ ]:
df_sem_out

In [ ]:
df_sem_out.info()

## Para saber mais: formas de identificar outliers

Existem várias técnicas para detectar outliers em um conjunto de dados. Algumas das mais comuns são: Z-score, regra dos 3 sigmas e a análise de dispersão.

**1) Z-score:**

O z-score é uma medida estatística que indica a quantos múltiplos do desvio-padrão um dado está distante da média. Para detectar outliers utilizando essa medida, basta calcular o z-score de cada dado e verificar se ele está muito distante da média. Se o z-score for maior do que um determinado limite, podemos considerar esse dado como um outlier.

O limite geralmente utilizado é de 3 desvios-padrões, ou seja, se o z-score de um dado for maior do que 3, podemos considerá-lo como um outlier.

```python

import numpy as np

# Dados de exemplo
data = np.array([10, 20, 30, 40, 150, 50, 60, 70, 80, 90, 100, 350])

# Cálculo do z-score
z_scores = (data - np.mean(data)) / np.std(data)

# Limite para considerar um dado como outlier
limite = 3

# Identificação dos outliers
outliers = data[np.abs(z_scores) > limite]

print("Outliers encontrados:", outliers)
```
**2) Regra dos 3 sigmas:**

A regra dos 3 sigmas é uma técnica que utiliza a distribuição normal para identificar outliers. Ela diz que cerca de 99,7% dos dados estarão dentro de 3 desvios padrão da média. Portanto, se um dado estiver a mais de 3 desvios padrão da média, podemos considerá-lo como um outlier.

```python

import numpy as np

# Criar um array com os dados
dados = np.array([0, 10, 12, 13, 15, 16, 18, 20, 22, 25, 30, 35, 40, 50, 350])

# Calcular a média e o desvio padrão do conjunto de dados
media = np.mean(dados)
desvio_padrao = np.std(dados)

# Definir o limite superior e inferior para identificar os outliers
limite_superior = media + (3 * desvio_padrao)
limite_inferior = media - (3 * desvio_padrao)

# Identificar os outliers no conjunto de dados
outliers = dados[(dados > limite_superior) | (dados < limite_inferior)]

print("Outliers:", outliers)
```
**3) Análise de dispersão**

A análise de dispersão é uma técnica que utiliza gráficos para identificar outliers. Um gráfico comumente utilizado é o boxplot que mostra a distribuição dos dados em quartis. Os outliers são identificados como pontos fora dos limites superior e inferior do boxplot.

```python

import matplotlib.pyplot as plt
import numpy as np

# Dados de exemplo
data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

# Criação do boxplot
fig, ax = plt.subplots()
ax.boxplot(data)

# Identificação dos outliers
outliers = data[(data < np.percentile(data, 25) - (1.5 * (np.percentile(data, 75) - np.percentile(data, 25)))) |
                (data > np.percentile(data, 75) + (1.5 * (np.percentile(data, 75) - np.percentile(data, 25))))]

print("Outliers encontrados:", outliers)
```

## **Trabalhando com variáveis categóricas**


#### **Variável Categórica**

- Forma de agrupar informacoes em categorias distintas, portanto, não consigo atribuir números para ela.
  Ex: Cor de olho: verde, castanho ou azul


In [ ]:
df_sem_out

##### Substituindo Valores


In [ ]:
# a coluna id_cliente nao é interessante manter pensando no nosso modelo de machine learning, pois seria mais uma coluna para ele tentar relacionar. Nesse caso, como sao valores unicos, nao faz sentido manter.
df_sem_id = df_sem_out.drop(columns='id_cliente', axis=1).copy()
df_sem_id

as colunas que estao com valores 0 ou 1, sao colunas que possuem variaveis categoricas binarias, ou seja, que possuem apenas duas categorias distintas.


In [ ]:
# para poder substituir os valores e alterar para o padrao de 0 e 1, iremos fazer uma mapeamento das informacoes que queremos substituir

mapeamento = {
    'nao': 0,
    'sim': 1,
    'masculino': 0,
    'feminino': 1,
}

In [ ]:
for col in df_sem_id.columns:
    print(f'Coluna: {col}')
    print(df_sem_id[col].unique())
    print('-' * 80)

In [ ]:
# portando com o codigo acima, conseguimos visualizar as colunas que sao do tipo binarias e vamos mapear atribuindo a uma variavel
colunas = ['telefone.servico_telefone', 'Churn', 'cliente.parceiro', 'cliente.dependentes', 'conta.faturamente_eletronico', 'cliente.genero']

In [ ]:
df_sem_id[colunas] = df_sem_id[colunas].replace(mapeamento).astype(int)
df_sem_id

In [ ]:
for col in df_sem_id.columns:
    print(f'Coluna: {col}')
    print(df_sem_id[col].unique())
    print('-' * 80)

#### **Para saber mais: o que são variáveis categóricas?**

As **variáveis categóricas** são um tipo de variável usado em Ciência de Dados para representar dados que podem ser classificados em diferentes categorias ou grupos. Por exemplo, se uma pessoa cientista de dados estiver analisando o desempenho acadêmico de estudantes em uma escola, ela pode usar variáveis categóricas para classificar o desempenho dessas pessoas em diferentes grupos, como "excelente", "bom" ou "regular". Isso permite identificar padrões ou tendências no desempenho.

Existem muitos exemplos de variáveis categóricas na Ciência de Dados. Algumas das mais comuns incluem a cor dos olhos, o tipo sanguíneo, a marca de um carro ou a escolaridade. Essas variáveis não possuem uma escala numérica e não podem ser medidas em termos de magnitude ou intensidade. Em vez disso, elas são usadas para classificar dados em diferentes grupos ou categorias.

Dentre os tipos de variáveis, temos:

**- Variáveis categóricas nominais**

São aquelas que não possuem uma ordem ou hierarquia específica entre as categorias. Por exemplo, se uma pessoa cientista de dados estiver analisando a preferência musical de um grupo de pessoas, ela pode usar variáveis categóricas nominais para classificar as pessoas em diferentes grupos, como: "rock", "jazz" ou "pop".

**- Variáveis categóricas ordinais**

Possuem uma ordem específica entre as categorias. Por exemplo, se a pessoa cientista de dados estiver analisando a escolaridade de um grupo de pessoas, ela pode usar esse tipo de variável para classificar as pessoas em diferentes grupos, como: "ensino fundamental completo", "ensino médio completo" ou "ensino superior completo" e assim por diante.

**- Variáveis categóricas binárias**

São um tipo especial de variável categórica que possui apenas duas categorias possíveis, por exemplo: sim/não, verdadeiro/falso ou presente/ausente. As variáveis categóricas binárias são úteis porque permitem que cientistas de dados analisem a distribuição de dados em apenas duas categorias possíveis.



#### **Onde Hot Enconder (Dummy)**

Com a **versão 2.0.0 do Pandas**, lançada em 03 de abril de 2023, houve uma mudança no valor padrão/default do parâmetro dtype do [método get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) . Ele passou de np.uint8 para bool.

Mas o que isso implica nas saídas que obtemos? Isso significa que você vai obter True e False ao invés de 1 e 0, caso esteja usando a versão 2.0.0 ou superior do Pandas, como no exemplo abaixo:

```python

s = pd.Series(list('abca'))
pd.get_dummies(s)Copiar código

```
**Saída**

<table>
    <thead>
        <tr>
            <th></th>
            <th>a</th>
            <th>b</th>
            <th>c</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0</td>
            <td>True</td>
            <td>False</td>
            <td>False</td>
        </tr>
        <tr>
            <td>1</td>
            <td>False</td>
            <td>True</td>
            <td>False</td>
        </tr>
        <tr>
            <td>2</td>
            <td>False</td>
            <td>False</td>
            <td>True</td>
        </tr>
        <tr>
            <td>3</td>
            <td>True</td>
            <td>False</td>
            <td>False</td>
        </tr>
    </tbody>
</table>


Essa mudança não tem implicações práticas para o propósito do projeto desse curso, que é preparar a base de dados para ser inserida em um modelo de machine learning, pois True e False são essencialmente equivalentes a 1 e 0, respectivamente. Portanto, ao criar variáveis dummies em Python, você pode usar True e False (sem precisar converter explicitamente para 1 e 0) e deixar as variáveis dummies com True e False sem problemas.

Caso sua intenção seja obter exatamente o mesmo resultado mostrado instrutor, basta passar o valor int para o parâmetro dtype da seguinte forma:

```python

import pandas as pd
s = pd.Series(list('abca'))
pd.get_dummies(s, dtype=int)  # aqui inserir o valor para o dtypeCopiar código

```
**Saída**

<table>
    <thead>
        <tr>
            <th></th>
            <th>a</th>
            <th>b</th>
            <th>c</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0</td>
            <td>1</td>
            <td>0</td>
            <td>0</td>
        </tr>
        <tr>
            <td>1</td>
            <td>0</td>
            <td>1</td>
            <td>0</td>
        </tr>
        <tr>
            <td>2</td>
            <td>0</td>
            <td>0</td>
            <td>1</td>
        </tr>
        <tr>
            <td>3</td>
            <td>1</td>
            <td>0</td>
            <td>0</td>
        </tr>
    </tbody>
</table>

Caso você queira conhecer mais do método, deixamos como sugestão de leitura complementar a documentação do método [get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

In [ ]:
s = pd.Series(list('abca'))
s

In [ ]:
pd.get_dummies(s)

In [ ]:
pd.get_dummies(s, dtype=int)

In [ ]:
df_sem_id.info()

In [ ]:
df_dummies = pd.get_dummies(df_sem_id, dtype=int).copy()
df_dummies

In [ ]:
df_dummies.columns

In [ ]:
df_dummies.info()

#### **Para saber mais: parâmetros do get_dummies()**

O [método get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) da biblioteca Pandas é utilizado para transformar variáveis categóricas em variáveis binárias. Abaixo estão os parâmetros do método:

1. `data`: **parâmetro obrigatório** que representa o conjunto de dados e contém as variáveis categóricas a serem transformadas em variáveis binárias.

2. `prefix`: é um parâmetro opcional utilizado para adicionar um prefixo às colunas binárias geradas pelo método `get_dummies()`. Por exemplo, se você definir o prefixo como "cat_", as colunas binárias geradas terão nomes como "cat_1", "cat_2", etc.

3. `prefix_sep`: parâmetro opcional utilizado para definir o separador entre o prefixo e o nome original da coluna categórica. O valor padrão é "_".

4. `columns`: parâmetro opcional utilizado para selecionar as colunas específicas do conjunto de dados que devem ser transformadas em variáveis binárias. Se não for especificado, todas as colunas categóricas serão transformadas.

5. `drop_first`: é um parâmetro opcional utilizado para remover a primeira coluna binária gerada pelo método `get_dummies()`. Isso é feito para evitar a multicolinearidade, que é uma situação em que duas ou mais variáveis independentes estão altamente correlacionadas entre si.

6. `dtype`: parâmetro opcional utilizado para definir o tipo de dado das colunas binárias geradas pelo método `get_dummies()`. O valor padrão é "uint8".

Abaixo está um exemplo de como podemos aplicar esses parâmetros:

```python

import pandas as pd

# Criando um DataFrame de exemplo
df = pd.DataFrame({'cor': ['vermelho', 'azul', 'verde', 'vermelho'],
                   'tamanho': ['pequeno', 'médio', 'grande', 'médio'],
                   'formato': ['quadrado', 'redondo', 'redondo', 'quadrado']})

# Transformando as colunas categóricas em variáveis numéricas binárias
df_dummies = pd.get_dummies(df, columns=['cor', 'tamanho'], prefix=['cor', 'tam'], prefix_sep='-', drop_first=True)

# Exibindo o DataFrame resultante
df_dummies
```

**Saída:**

Neste exemplo, o parâmetro `data` é o DataFrame `df` que contém as colunas categóricas a serem transformadas. O parâmetro `columns` é uma lista que contém os nomes das colunas categóricas a serem transformadas em variáveis binárias. O parâmetro `prefix` é uma lista que contém os prefixos a serem adicionados às colunas binárias geradas. Aqui, estamos adicionando o prefixo "cor_" às colunas binárias geradas a partir da coluna "cor" e o prefixo "tam_" às colunas binárias geradas a partir da coluna "tamanho".

Adicionamos também os parâmetros `prefix_sep='-'` para definir o separador entre o prefixo e o nome original da coluna categórica como um hífen (-). Também adicionamos o parâmetro `drop_first=True` para remover a primeira coluna binária gerada pelo método get_dummies(). Isso é feito para evitar a multicolinearidade, que é uma situação em que duas ou mais variáveis independentes estão altamente correlacionadas entre si.

O resultado será um novo DataFrame chamado `df_dummies`, que contém as colunas originais do DataFrame `df`, bem como as novas colunas binárias geradas pelo método `get_dummies()` com um hífen (-) como separador de prefixo e a primeira coluna binária removida.